In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import urllib2
from __future__ import division

plt.style.use('ggplot')
np.random.seed(1)
url = ('https://raw.githubusercontent.com/Upward-Spiral-Science'
       '/data/master/syn-density/output.csv')
data = urllib2.urlopen(url)
csv = np.genfromtxt(data, delimiter=",",dtype='int')[1:] # don't want first row (labels)
data = csv
print data[::1000]
sizes = [len(np.unique(data[:, i])) for i in range(3)]
ranges = [(np.max(data[:, i]), np.min(data[:,i])) for i in range(3)]
ranges_diff = [np.max(data[:, i])-np.min(data[:,i]) for i in range(3)]
print np.max(data[:,3])
print sizes
print ranges
print ranges_diff
for i, ax in zip(range(3), ['x', 'y', 'z']):
    print ax + '-axis: '
    print 'unique bins (in data): ', np.unique(data[:, i]).size
    print 'bin length: ', np.unique(data[:, i])[1]-np.unique(data[:, i])[0]
    print np.unique(data[:, i])
    print

[[    19   1369     55   5063      0]
 [    58   2851   1165      0      0]
 [   136   2344   1054   3042      0]
 [   214   1837    943   3224      0]
 [   253   3358    832      0      0]
 [   331   2851    721  82193     87]
 [   409   2344    610 118055    147]
 [   487   1837    499 153461    205]
 [   526   3358    388      0      0]
 [   604   2851    277  61430     20]
 [   682   2344    166 132042    177]
 [   760   1837     55 153350    177]
 [   799   3319   1165  15316      7]
 [   877   2812   1054 152682    194]
 [   955   2305    943 138771    286]
 [  1033   1798    832 134847    186]
 [  1072   3319    721      0      0]
 [  1150   2812    610 126243    115]
 [  1228   2305    499 150331    178]
 [  1306   1798    388 150579    263]
 [  1345   3319    277      0      0]
 [  1423   2812    166 134814     83]
 [  1501   2305     55 133161    132]
 [  1579   1759   1165 159705    207]
 [  1618   3280   1054   5186      0]
 [  1696   2773    943 152424    120]
 [  1774   2

Seems that the coordinates in our data set line up with the viz.neurodata coordinates at resolution 5. To demonstrate why this seems likely to be true, markers have been placed at (0, 0), (4192, 0), (0, 3358), (4192, 3358)--i.e. corners of the boundary--and an image of these markers at resolution 8 is shown below.

<img src="viz_bounds.png" style="width:400px">

Inspecting the JS/html more we see that the actual number of pixels at resolution 5 is 4232x3744, and as we decrease resolution the pixels are doubled. Assuming the data coordinates mark bin centers (to explain the offset of the initial x). 

In [118]:
# make coordinates to bin boundaries mapping
def cdata_to_pixels(cx, cy, res, binwidthx=0, binwidthy=0):
    bounds = [(c-19, c+(39*w)+20) for c,w in [(cx, binwidthx), (cy, binwidthy)]]
    bounds = np.array(bounds)
    power = 5-res
    return bounds*(2**power)

xMax, yMax = [np.max(data[:, i]) for i in [0, 1]]
print xMax, yMax
for res in range(6):
    print cdata_to_pixels(xMax, yMax, res)

4192 3358
[[133536 128544]
 [106848 101856]]
[[66768 64272]
 [53424 50928]]
[[33384 32136]
 [26712 25464]]
[[16692 16068]
 [13356 12732]]
[[8346 8034]
 [6678 6366]]
[[4173 4017]
 [3339 3183]]


Now just need to figure out z-axis. The z values in the image data go from 2917-4156, which is a range of 

In [47]:
print (2917-4156)*-1

1239


So it seems that the z-values in the data correspond approximately to the z-values in the image data, other than a translation of 2917.

### Grabbing images from website
Looking through the JavaScript code on viz.neurodata.io, we see that its tiling with 512x512 .png images, and that the actual pixel resolution and resolution according to the website correspond.

In [265]:
from IPython.display import Image, HTML, display


def get_bin_image(cx, cy, xwidth=0, ywidth=0, cz=55, res=1, imsize=(512,512), disp_text=False):
    """Return the image of the bin corresponding to cx, cy, cz in the data
    params:
        cx, cy, xwidth, ywidth, cz - data coordinates/bin widths
        res - resolution to get image at
        imsize - width/height to output at
    returns:
        HTML for generating image, canvas id
    """
    z = 2917
    cx, cy = int(cx), int(cy)
    bounds = cdata_to_pixels(cx, cy, res, xwidth, ywidth)
    size = bounds[:, 1]-bounds[:,0]
    
    tiles = bounds//512
    # tiles = tiles + [[0, 1],[0, 1]]
    crops = np.empty((2, 2))
    crops = bounds-(tiles*512)
    # crops = crops.T
    tiles = tiles + [[0, 1],[0, 1]]
    imgtags = []
    ids = []
    cargs = {}
    for k, y in enumerate(np.arange(*tiles[1, :])):
        for i, x in enumerate(np.arange(*tiles[0,:])):
                width, height = [512]*2
                sx, sy, dx, dy = [0]*4
                if i==0:
                    # imgtags.append("<div> \n")
                    sx = crops[0, 0]
                    width=512 - crops[0, 0]
                elif i==len(np.arange(*tiles[1, :]))-1:
                    sx = 0
                    width=crops[0,1]
                if k==0:
                    height=512 - crops[1, 0]
                    sy = crops[1, 0]
                elif k==len(np.arange(*tiles[1, :]))-1:
                    sy = 0
                    height = crops[1, 1]
                newargs = {'sx':sx, 'sy':sy, 
                           'swidth':width, 'sheight':height,
                           'dx':0, 'dy':0,
                           'dwidth':width, 'dheight':height}
                end = '/'+reduce(lambda x, y: str(x) +'_'+str(y), [y, x, res])
                end += '.png'
                newid = str(i)+'-'+str(k)
                ids += [newid]
                cargs[newid] = newargs
                imgurl = 'http://openconnecto.me/ocp/catmaid/bock11/image/xy/'+str(z) +end
                imgtags.append("<img crossOrigin='Anonymous' src='%s' id='%s' style='display: none'/>" % (imgurl, newid))
    for xyid in ids:
        x, y = [int(i) for i in xyid.split('-')]
        if x > 0:
            prev = cargs[str(x-1)+'-'+str(y)]
            cargs[xyid]['dx'] += prev['dwidth']+prev['dx']
        if y > 0:
            prev = cargs[str(x)+'-'+str(y-1)]
            cargs[xyid]['dy'] += prev['dheight']+prev['dy']
    canvasID = int(np.random.ranf()*9999999)
    javascript = """
    <script>
        var c = document.getElementById('%s');
        c.width = %s;
        c.height = %s;
        var ctx = c.getContext("2d");
    """ % tuple([canvasID]+ list(size))
    for xyid in ids:
        javascript += "var img = document.getElementById('%s');\n" % xyid
        javascript += """
        ctx.drawImage(img, {sx}, {sy}, {swidth}, {sheight}, {dx}, {dy}, {dwidth}, {dheight} );
        """.format(**cargs[xyid])
            
    javascript += """
    var tempCanvas = document.createElement('canvas');
    tempCanvas.width = c.width;
    tempCanvas.height = c.height;
    tempCanvas.getContext('2d').drawImage(c, 0, 0);
    c.width = %s;
    c.height = %s;
    c.getContext('2d').drawImage(tempCanvas, 0, 0, tempCanvas.width, tempCanvas.height, 0, 0, c.width, c.height);
    """ % imsize

    javascript += '</script>'
    html = "<canvas id='%s'>" % canvasID
    html += "</canvas>"
    html += reduce(lambda x, y: x+'\n'+y, imgtags)+javascript
    html_raw = html
    html = HTML(html)
    display(html)
    return html_raw, canvasID
    
html_raw,_ = get_bin_image(19, 2188, 10, 10, imsize=(1000,1000))
print html_raw

<canvas id='4902534'></canvas><img crossOrigin='Anonymous' src='http://openconnecto.me/ocp/catmaid/bock11/image/xy/2917/67_0_1.png' id='0-0' style='display: none'/>
<img crossOrigin='Anonymous' src='http://openconnecto.me/ocp/catmaid/bock11/image/xy/2917/67_1_1.png' id='1-0' style='display: none'/>
<img crossOrigin='Anonymous' src='http://openconnecto.me/ocp/catmaid/bock11/image/xy/2917/67_2_1.png' id='2-0' style='display: none'/>
<img crossOrigin='Anonymous' src='http://openconnecto.me/ocp/catmaid/bock11/image/xy/2917/67_3_1.png' id='3-0' style='display: none'/>
<img crossOrigin='Anonymous' src='http://openconnecto.me/ocp/catmaid/bock11/image/xy/2917/67_4_1.png' id='4-0' style='display: none'/>
<img crossOrigin='Anonymous' src='http://openconnecto.me/ocp/catmaid/bock11/image/xy/2917/67_5_1.png' id='5-0' style='display: none'/>
<img crossOrigin='Anonymous' src='http://openconnecto.me/ocp/catmaid/bock11/image/xy/2917/67_6_1.png' id='6-0' style='display: none'/>
<img crossOrigin='Anonymo

In [116]:
uniX, uniY = [np.unique(data[:,i]) for i in (0, 1)]
html = ''
for x in uniX[:50]:
    h = get_bin_image(x, np.median(uniY), imsize=100, render=False)
    h = "<div style='float: left;'>"+h+"</div>"
    html += h
display(HTML(html))

In [262]:
'67072-66560'.split('-')

['67072', '66560']

In [ ]:
bin_size = cdata_to_pixels(19, 19, res)
    bin_size = bin_size[:,1][0]
    bin_coords = np.array([np.arange(xwidth)*bin_size, np.arange(ywidth)*bin_size]).T
    bin_size_real = cdata_to_pixels(19,19,5)[:,1][0]
    real_coords = np.array([np.arange(xwidth)*bin_size_real, 
                            np.arange(ywidth)*bin_size_real]).T+[cx, cy]
    print real_coords[0]
    mesh = np.array(np.meshgrid(real_coords[0], real_coords[1], indexing='ij'))
    s_values = np.empty((xwidth, ywidth))
    u_values = np.empty((xwidth, ywidth))
    for x in range(xwidth):
        for y in range(ywidth):
            m = mesh[:, x, y]
            a = data[:, (0, 1)]==m
            a = np.logical_and(a[:,0], a[:,1])
            d = data[a,:]
            a = d[:,2]==cz
            print cz
            print d[a,4]
            s_values[x,y] = d[a,4]
            u_values[x,y] = d[a,3]
    print s_values
    print u_values
    mesh = np.array(np.meshgrid(bin_coords[0], bin_coords[1], indexing='ij'))
    print mesh.shape
    for x in range(xwidth):
        for y in range(ywidth):
            print mesh.shape
            xx = mesh[0,x,y]
            yy = mesh[1,x,y]
            s = 'synapses: ' + str(s_values[x, y])
            s1 = 'unmasked: ' + str( u_values[x, y])
            print s
            javascript += """
            ctx.font = "50px Arial";
            ctx.fillText('%s', '%s', '%s');
            ctx.fillText('%s', '%s', '%s');
            """ % (s, xx, yy+100, s1, xx, yy+300)